In [12]:
%pwd

'd:\\CDAC\\RSN\\PROJECT\\Medical-AI-Chatbot\\research'

In [18]:
import os
os.chdir("../")

In [19]:
%pwd

'd:\\CDAC\\RSN\\PROJECT\\Medical-AI-Chatbot'

# pdf data extraction

In [20]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [21]:
def load_pdf_file(data):
    loader = DirectoryLoader(data, glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents    

In [22]:
extracted_data = load_pdf_file(data ='Data/')

In [23]:
len(extracted_data)

637

# split the data into text chunk

In [24]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [25]:
text_chunks =text_split(extracted_data)
len(text_chunks)

5859

# Embedding from HF

In [26]:
from langchain.embeddings import HuggingFaceEmbeddings

In [27]:
def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [28]:
embeddings = download_hugging_face_embedding()

C:\Users\Roshith\AppData\Local\Temp\ipykernel_20684\1183320502.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [29]:
a = embeddings.embed_query("hallo")
len(a)

384

In [54]:
from dotenv import load_dotenv
load_dotenv()

True

In [55]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")

In [56]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)


index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-v45evji.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [37]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY

# EMBED EACH CHUNKS AND INSERT EMBEDDINDS TO PINECONE DATABASE

In [57]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding=embeddings
)

In [58]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding=embeddings
)

docsearch

In [59]:
retriever = docsearch.as_retriever(search_type ="similarity", search_kwargs={"k":5})

In [60]:
retrieved_answer = retriever.invoke("what is pimples")

In [61]:
retrieved_answer

[Document(id='a47894e0-4c39-400e-873e-865869289fff', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 37.0, 'page_label': '38', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\medical_book.pdf', 'total_pages': 637.0}, page_content='skin. A pimple forms when the damaged follicle weakens\nand bursts open, releasing sebum, bacteria, and skin and\nwhite blood cells into the surrounding tissues. Inflamed\npimples near the skin’s surface are called papules; when\ndeeper, they are called pustules. The most severe type of\nacne consists of cysts (closed sacs) and nodules (hard\nswellings). Scarring occurs when new skin cells are laid\ndown to replace damaged cells.\nThe most common sites of acne are the face, chest,'),
 Document(id='f2c92462-8777-4249-a34a-72fc053fe7da', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 37.0, 'page_label': 

In [47]:
# from langchain_openai import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=500)

from langchain.chat_models import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv()
together_api_key = os.getenv("TOGETHER_API_KEY")

llm = ChatOpenAI(
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
,  # You can use other supported models too
    openai_api_base="https://api.together.xyz/v1",
    openai_api_key=together_api_key,
    temperature=0.4,
    max_tokens=500,
)


In [48]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [49]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [53]:
response = rag_chain.invoke({"input": "what is gigantism and how to cure?"})
print(response["answer"])

 Gigantism is a condition of exceptional growth in children, caused by an abnormal release of growth hormone (GH) from the pituitary gland before the bones have stopped growing. It is a variant of acromegaly. The text does not provide information on how to cure gigantism. However, acromegaly and gigantism are typically treated by controlling the excess GH, often through surgery, medication, or radiation therapy. Please consult a healthcare professional for medical advice.


In [62]:
import streamlit as st
from dotenv import load_dotenv
import os

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Load API keys
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

st.write(f"Pinecone Key Loaded: {PINECONE_API_KEY is not None}")
st.write(f"Together Key Loaded: {TOGETHER_API_KEY is not None}")


# Set up LLM (Together AI)
llm = ChatOpenAI(
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
    openai_api_base="https://api.together.xyz/v1",
    openai_api_key=TOGETHER_API_KEY,
    temperature=0.4,
    max_tokens=500,
)

# Load PDFs and create vector DB (only once)
# @st.cache_resource
def setup_vector_store():
    loader = DirectoryLoader('data/', glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    documents = loader.load()
    st.write(f"Loaded {len(documents)} documents.")

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    chunks = splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

    pc = Pinecone(api_key=PINECONE_API_KEY)
    index_name = "medicalbot"
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=384,
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )

    vectorstore = PineconeVectorStore.from_documents(
        documents=chunks,
        index_name=index_name,
        embedding=embeddings
    )
    return vectorstore

docsearch = setup_vector_store()
st.write("Initializing vector store...")

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Setup RAG chain
system_prompt = (
    "You are an assistant for medical question-answering tasks. "
    "Use the following pieces of retrieved context to answer. "
    "If you don't know the answer, say so. "
    "Use concise language, maximum three sentences.\n\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])
qa_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, qa_chain)

# UI
st.title("Medical AI Chatbot")
st.write("Ask any medical question based on the uploaded documents.")

user_input = st.text_input("Enter your medical question here...")

if user_input:
    with st.spinner("Thinking..."):
        response = rag_chain.invoke({"input": user_input})
        st.success(response["answer"])


2025-05-15 11:49:36.300 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 11:49:36.411 
  command:

    streamlit run d:\CDAC\RSN\PROJECT\Medical-AI-Chatbot\medibot\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-05-15 11:49:36.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 11:49:36.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 11:49:36.413 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 11:50:04.280 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 11:50:04.289 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-05-15 11:51:52.440 T